In [227]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

# Business Problem

A Car insurance company wants to predict the severity of an accident based on the features of the car and other usage pattern of the applicant. The accurate prediction will help the company categorise users based on the chances of an applicant getting into severe accidents and further decision making like identifying high risk/low risk customers and offering a customized insurance premium to them.

The company will have the following features of the vehicle and details of the applicant in during the application process. The goal is to create a predictive model for the company using these features to predict accident severity.

**Independent Variables**:

1. Vehicle Type
2. Journey purpose of the driver
3. Sex of the driver
4. Age of the driver
5. Engine capacity. of the vehicle (cc)
6. Propulsion code
7. Age of vehicle
8. Generic make model of the vehicle
9. Driver's IMD decile
10. Driver's Home area type
11. Vehicle left hand drive

**Dependent Variable**: Accident Severity\



**Dataset Information**:

We are going to use `Accident` and `Vehicle` data of the year 2020 from the website of govt of UK from [UK Road Safety Data](https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data "Click to see the source"). Firt we will select the relevant columns from both the data and then we will merge the two using the column `accident_index` which is a unique value for each accident.

In [228]:
import io
from nbformat import read, NO_CONVERT

with io.open(NOTEBOOK_FULL_PATH.split("/")[-1], 'r', encoding='utf-8') as f:
    nb = read(f, NO_CONVERT)

word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print(f"Word count: {word_count}")

Word count: 1482


# Data collection

In [ ]:
# setting logging to print only error messages from Sklearnex
import logging
logging.basicConfig()
logging.getLogger("SKLEARNEX").setLevel(logging.ERROR)

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

sns.set_theme(palette="Set2")

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None) #to see all the columns of the dataframe

In [ ]:
import os
import tarfile
import urllib


# urls to download the data
URL1 = "https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-accident-2020.csv"
URL2 = "https://data.dft.gov.uk/road-accidents-safety-data/dft-road-casualty-statistics-vehicle-2020.csv"
URL3 = "https://data.dft.gov.uk/road-accidents-safety-data/Road-Safety-Open-Dataset-Data-Guide.xlsx"

#function to load the dataframes
def get_dataframe():
    global URL1,URL2
    
    # if the "datasets" folder does not exist, create it
    if not os.path.exists("datasets"):
        os.makedirs("datasets")
    
    # if the accident file does not exist, download it
    if not os.path.exists("datasets/accident2020.csv"):
        urllib.request.urlretrieve(URL1, "datasets/accident2020.csv")
     
    # if the vehicle file does not exist, download it
    if not os.path.exists("datasets/vehicle2020.csv"):
        urllib.request.urlretrieve(URL2, "datasets/vehicle2020.csv")   
        
    # if the schema file does not exist, download it
    if not os.path.exists("datasets/schema.csv"):
        urllib.request.urlretrieve(URL3, "datasets/schema.xlsx")
        
    ac = pd.read_csv("datasets/accident2020.csv")
    vh = pd.read_csv("datasets/vehicle2020.csv")
    schema = pd.read_excel("datasets/schema.xlsx")
    
    # load the dataframe
    return ac,vh,schema

In [ ]:
#loading the data
ac,vh,schema = get_dataframe()

In [ ]:
#checking accident df
ac.head()

In [ ]:
#checking vehicle df
vh.head()

In [ ]:
#checking the schema
schema.head()

In [ ]:
schema.columns

In [ ]:
#setting field name as index to acess the fields easily

schema.set_index("field name",inplace=True)

In [ ]:
schema.head()

In [ ]:
#example after setting index
schema.loc["vehicle_reference"].note[0]

- Similarly we can access the values from the schema to understand the data more.

In [ ]:
#checking shape of both dataframes

print("Shape of the Accident dataframe is", ac.shape)
print("Shape of the Vehicle dataframe is", vh.shape)


- The number of rows in Vehicle dataset is more than 1.5 times of the same in Accident dataset. Lets explore more to see what is causing it since both the datasets are from the year 2020.

In [ ]:
#checking unique accident index in ac & vh

if ac.accident_index.nunique() == ac.shape[0]:
    print("All values in accident_index are unique in Accident dataframe.")
else:
    print("All values in accident_index are not unique in Accident dataframe.")
    
if vh.accident_index.nunique() == vh.shape[0]:
    print("All values in accident_index are unique in Vehicle dataframe.")
else:
    print("All values in accident_index are not unique in Vehicle dataframe.")

- This means we have multiple records of each accident in the vehicle data. Lets look at both the dataframes to confirm this.

In [ ]:
#checking initial rows of ac
ac.head(10)

In [ ]:
#checking initial rows of vh
vh.head(10)

- By looking at both the dataframes we can see that some of the accident index has multiple rows in vehicle. We can see the vehicle reference has multiple values for a single accident index, which shows the details of multiple vehicles involved a single accident. This increases the number of rows in the vehicle dataset which is good for our problem statement as we will be able to use different vehicle data while training the predictive model.

## 1.1 Filtering Data

In [ ]:
#Selecting required columns

#Selecting required cols from accident 
ac = ac[['accident_index','longitude',
        'latitude','accident_severity']]

#selecting required cols from vehicle
vh = vh[['accident_index','vehicle_type','journey_purpose_of_driver','sex_of_driver',
       'age_of_driver','age_band_of_driver','engine_capacity_cc',
        'propulsion_code','age_of_vehicle','driver_home_area_type','driver_imd_decile',
        'vehicle_left_hand_drive']]

In [ ]:
#selecting vh type only for cars

vh.vehicle_type.unique()

In [ ]:
#looking at the schema to understand the types

schema.loc["vehicle_type"]

In [ ]:
vh.vehicle_type.value_counts()

- The vehicle type 9 has the highest number of observations which is what we need for the analysis.

- We could have merged type 108 and 109 too which also represent cars but we don't have records for those type.

In [ ]:
#selecting data only for car

vh = vh[vh["vehicle_type"] == 9]
vh.shape

In [ ]:
vh.vehicle_type.value_counts()

- We don't need the variable vehicle_type anymore since all the records belong to car now.

In [ ]:
#Dropping vehicle_type

vh.drop("vehicle_type",axis=1,inplace=True)


In [ ]:
#checking the shape after dropping
vh.shape

## Merging two data frames

In [ ]:
#merging the dataframes
import functools as ft

dfs = [ac,vh]

df_merged = ft.reduce(lambda left, right: pd.merge(left, right, on='accident_index'), dfs)

In [ ]:
df_merged.head()

In [ ]:
df_merged.shape

## Data Insepction

Let's look at the schema which explains each of the variable in out data.

In [ ]:
schema

In [ ]:
#Understanding the target variable

schema.loc["accident_severity"]

In [ ]:
#journey_purpose_of_driver

schema.loc['journey_purpose_of_driver']

In [ ]:
#driver_home_area_type
schema.loc["driver_home_area_type"]

- As we see '-1' represents data missing or out of range in many variables. These must be removed as these are not valid.

In [ ]:
df_merged.shape

In [ ]:
#Let's remove all rows having -1

df_merged = df_merged.replace(-1, np.nan).dropna(axis=0)
df_merged.shape

In [ ]:
df_merged.head()


- Many categorical variables have been stored as integer/float type. Most of them are represented by numbers and these are not ordinal.We are going to replace those numbers with the respective category as described in the schema. It will also help in unique columns after we change these to dummy variables.

### Changing label for categorical variables

In [ ]:
# function to change labels for categorical variables

def change_label(col):
    df_merged[col] = df_merged[col].replace(schema.loc[col].set_index('code/format')['label'])

In [ ]:
# #changing labels for categorical variables

cat_cols = ['accident_severity','journey_purpose_of_driver','sex_of_driver','age_band_of_driver',
        'propulsion_code','driver_home_area_type','driver_imd_decile',
        'vehicle_left_hand_drive']
#df_merged[cols] = df_merged[cols].astype('object')

for col in cat_cols:
    change_label(col)

In [ ]:
#df_merged[cols] = df_merged[cols].astype('category')

In [ ]:
#dropping accident_index as it is not required anymore
df_merged.drop('accident_index',axis=1,inplace=True)

In [ ]:
df_merged.info()

# Feature Engineering

- We will cluster the lattitude and longitude data to divide into different locations and will add a new variable named location to use it as a predictor.

- We will divide the whole of UK into 4 regions and see if it makes sense to add an additional variable.

## K-Means clustering of longitude and latitude 

In [ ]:
#putting latitude and longitude in a new dataframe
df = df_merged[['latitude','longitude']]


In [ ]:
# #Calculating the Hopkins statistic
# from sklearn.neighbors import NearestNeighbors
# from random import sample
# from numpy.random import uniform
# import numpy as np
# from math import isnan
 
# def hopkins(X):
#     d = X.shape[1]
#     #d = len(vars) # columns
#     n = len(X) # rows
#     m = int(0.1 * n) 
#     nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
#     rand_X = sample(range(0, n, 1), m)
 
#     ujd = []
#     wjd = []
#     for j in range(0, m):
#         u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
#         ujd.append(u_dist[0][1])
#         w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
#         wjd.append(w_dist[0][1])
 
#     H = sum(ujd) / (sum(ujd) + sum(wjd))
#     if isnan(H):
#         print(ujd, wjd)
#         H = 0
 
#     return H

In [ ]:
# hopkins(df)

In [ ]:
# from sklearn.cluster import KMeans

# #Lets do the silhouette score analysis to find the right number of clusters
# from sklearn.metrics import silhouette_score
# sse_ = []
# for k in range(2, 5):
#     kmeans = KMeans(n_clusters=k).fit(df)
#     sse_.append([k, silhouette_score(df, kmeans.labels_)])

In [ ]:
# #plotting silhouette_score vs number of clusters
# plt.plot(pd.DataFrame(sse_)[0], pd.DataFrame(sse_)[1]);

- The analysis shows 3 is the right number of clusters.

In [ ]:
# K-Means using K=3

from sklearn.cluster import KMeans
model_clus3 = KMeans(n_clusters = 3, max_iter=50,random_state = 50)
model_clus3.fit(df)

In [ ]:
# Assigning Cluster IDs

df.index = pd.RangeIndex(len(df.index))
df = pd.concat([df, pd.Series(model_clus3.labels_)], axis=1)
df.columns = ['latitude', 'longitude','cluster_id']
df.head()

In [ ]:
df.info()

In [ ]:
#checking counts
df.cluster_id.value_counts()

In [ ]:
#changing data type
df.cluster_id = df.cluster_id.astype('object')

In [ ]:
#plotting clusters
import plotly.express as px

fig = px.scatter(df, x='longitude', y='latitude',color='cluster_id')
fig.show()

- As per the above plot it is fair to divide the whole of UK into 3 different regions. The Insurance company can get the location from the customers and can place them in the appropriate category.

- Cluster 1 represents south-east, 2 represents south west, and 0 represents rest of UK. 

In [ ]:
test = df_merged

In [ ]:
#merging clusterid to original data frame

df_merged['location'] = df['cluster_id'].values

In [ ]:
df_merged.info()

In [ ]:
# changing location id to region names
df_merged['location'] = df_merged['location'].map({0:'Rest of UK',1:'South-east',
                                                   2:'South-west'})

In [ ]:
df_merged['location'].value_counts()

In [ ]:
fig = px.scatter(df_merged, x='longitude', y='latitude',color='location')
fig.show()

# Train-Test Split

In [ ]:
df_merged.shape

In [ ]:
#checking count of the target variable
df_merged.accident_severity.value_counts()

In [ ]:
#splitting the set with similar proportion of accident_severity

from sklearn.model_selection import train_test_split

train_df,test_df = train_test_split(df_merged, test_size=0.2, random_state=7,
                                    stratify=df_merged["accident_severity"])

# train_df,test_df = train_test_split(df_merged, test_size=0.2, random_state=7,
#                                       stratify=df_merged["accident_severity"])

In [ ]:
print(f"{train_df.shape[0]} train and {test_df.shape[0]} test instances")

# EDA

## Descriptive Statistics & Visualization

In [ ]:
# #using pandas profiling for initial exploration
# from pandas_profiling import ProfileReport
# prof_rep = ProfileReport(train_df)
# prof_rep.to_file(output_file='output.html')

In [ ]:
# prof_rep

#### Insights from Pandas Profile Report

- Target variable accident_severity is highly imbalanced.
- Around 57% of rows in journey purpose of driver is unknown. We will have to clean it.
- Around 80% of drivers are from urban area. This might have a effect  which we will have to explore further.
- High correlations found among some variables which we will look separately.


### Checking the distribution of numeric variables 

In [ ]:
train_df.describe()

- The maximum age of driver is 98 which is very unlikely. We need to address this while cleaning.

- There is a huge gap between the minimum,maximum engine capacity and the median one. We will adress this in the data cleaning process.
- We are observing extreme values for age_of_vehicle too. Lets visualize all three.

In [ ]:
import plotly.figure_factory as ff


In [ ]:
#creating a function to create distribution of a continuous variable
import plotly.figure_factory as ff
import plotly.offline as pyo

def create_hist(var, group_label, bin_):
    
    fig = ff.create_distplot([var],group_labels = [group_label], curve_type = 'normal', 
                             bin_size = bin_)
    fig.show()

In [ ]:
# #Visualizing age of driver

# create_hist(train_df['age_of_driver'], 'Age of Driver',10)

- Minimum age is 11 in the dataset and some observations are less than 16 which is the legal age in UK. These must be removed.
- Some observations for age are close to 100, which we should address during data cleaning.

In [ ]:
# #Visualizing engine_capacity_cc

# create_hist(train_df['engine_capacity_cc'], 'Engine CApacity',100)

In [ ]:
# #Visualizing age of vehicle

# create_hist(train_df['age_of_vehicle'], 'Age of Vehicle',10)

- Most of the vehicles are less than 20 years of age. Some extreme values are also present.

#### Bi-variate analysis (Numeric variables)

In [ ]:
#Creating a function to plot scatter plots showing relationship between two continuous variables

import plotly.express as px

def create_scat(df, var1, var2):
    fig = px.scatter(df, x=var1, y=var2)
    fig.show()

In [ ]:
# #Comparing applicant_income & loan_amount

# create_scat(train_df,'age_of_driver','age_of_vehicle')



- The age of the driver and age of vehicle are not related. The data is evenly spread.

In [ ]:
# #plotting longitude and lattitude to see the  spread of accidents across location
# px.set_mapbox_access_token("pk.eyJ1Ijoic2FzaGlrYW50NyIsImEiOiJjbDRmbjFpOWswMG01M2RranZ4b2VrZDgxIn0.IDHMdhaaKIT73LTDRLym2A")
# fig = px.scatter_mapbox(train_df, lat="latitude", lon="longitude", color="accident_severity",
#                   color_continuous_scale=px.colors.cyclical.IceFire, size_max=20,zoom=4)
# fig.show()

- Most of the observations in our data point are from England. By zooming in we can see that observations from Wales and Scotland are very few.

- Different severity types are evenly spread across UK and our predictive model will be able to generalize well across UK which suits our requirement in the business problem

### Exploring the categorical variables

In [ ]:
#creating a function for categorical variables analysis

def cat_plot(col):
    print(train_df[col].value_counts()/len(train_df[col])*100) # % of categories
    
    fig = px.histogram(train_df, x=col,color='accident_severity',histnorm='percent',text_auto=True)
    fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
    fig.show()

In [ ]:
#checking journey_purpose_of_driver
cat_plot('journey_purpose_of_driver')

- 56.73% journey purpose is not known. We can not fix this so we will have to drop the whole column.
- It is interesting to see that in 62% of the fatal cases the journey purpose is unknown.

In [ ]:
#checking sex_of_driver
cat_plot('sex_of_driver')

- In 71.8% of the fatal accident cases the driver is a male.
- Female are morelikely to be involved in a slight accident.

In [ ]:
#checking age_band_of_driver
cat_plot('age_band_of_driver')

- In 24.69% of the accidents, age of the driver is between 26-35. 

- It is noticable that the age group 'over 75' is more involved in fatal accidents.

- We have few variables in the age group od 11-15 and 6-10. These are outiers which we must remove.

In [ ]:
#checking propulsion_code
cat_plot('propulsion_code')

- Petrol cars are involved in 57.64% of the total accidents. 57.91% of fatal accidents also caused by petrol cars.

- Other than Pentrol and diesel there are very few items for other fuel types. We will merge these to one group.

In [ ]:
#checking driver_imd_decile
cat_plot('driver_imd_decile')

- It is quite evident that drivers from less deprived areas cause more fatal accidents.

In [ ]:
#checking driver_home_area_type
cat_plot('driver_home_area_type')

- Around 80% of drivers involved in an accident come from urban areas. 

- Rural area drivers are more likely to get involved in a fatal accident.

In [ ]:
#checking vehicle_left_hand_drive
cat_plot('vehicle_left_hand_drive')

- 95% of the cars have right hand drive. It is better to drop the whole column since it doesn't add much variance to the dataset.

In [ ]:
#checking vehicle_left_hand_drive
cat_plot('location')

- Proportion of fatal accidents are more in South-west and Rest of the UK while it is less in the south east region.

In [ ]:
#checking target variable 'accident_severity'

fig = px.histogram(train_df, x='accident_severity',histnorm='percent',text_auto=True)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

- There is a huge class imbalance in our dataset. Number of accidents with slight category is the highest and fatal is the lowest. We should balance the dataset before the modelling stage.

### Correlation analysis for all variables

In pandas profile report we identified that there are some variables in our data set which are highly correlated. We will idenrify those using the **Phik (φk)** which is a new and practical correlation coefficient that works consistently between categorical, ordinal and interval variables, captures non-linear dependency and reverts to the Pearson correlation coefficient in case of a bivariate normal input distribution.  

In [ ]:
import phik
from phik import resources, report

In [ ]:
# get the phi_k correlation matrix between all variables
corr = train_df.phik_matrix()
mask = np.triu(np.ones_like(corr, dtype=bool))
corr = corr.mask(mask)
corr.fillna('', inplace=True)

In [ ]:
#plotting a heatmat
fig = px.imshow(corr, text_auto=True,aspect="auto")
fig.show()

- The variables age and age group of driver are highly correlated. We will drop age group of driver.Longitude and latitude are highly correlated too. We will drop both as we have already converted those to location column.

- Some other pairs like vehicle_left_hand_drive and sex_of_driver , make and engine_capacity, make and propulsion code have positive correlations but those are onder 0.60. So we will keep those.

- There is no risk of multicollinearity once we drop age_band_of_driver. 

# Data Cleaning & Transformation

Apart from adressing all the data isuues identified in the EDA stage, we will also look for other data issues and try to fix those.

In [ ]:
#checking null values

print("Sum of null values in training set is:",train_df.isna().sum())
print("Sum of null values in testing set is:",train_df.isna().sum())

In [ ]:
train_df.head()

In [ ]:
#dropping journey_purpose_of_driver,age_band_of_driver',
#'vehicle_left_hand_drive', 'longitude' & 'latitude' as decided in the EDA stage

#dropping from train
train_df.drop(['journey_purpose_of_driver','age_band_of_driver','longitude',
               'latitude','vehicle_left_hand_drive'],axis=1,inplace=True)

#dropping from test
test_df.drop(['journey_purpose_of_driver','age_band_of_driver','longitude',
               'latitude','vehicle_left_hand_drive'],axis=1,inplace=True)

In [ ]:
df_merged.propulsion_code.value_counts()

In [ ]:
#combining hybrid types to other in propulsion_code
vals = ['Hybrid electric', 'Petrol/Gas (LPG)','Gas/Bi-fuel', 'Electric diesel',
       'Gas', 'Gas Diesel']

train_df['propulsion_code'] = train_df['propulsion_code'].replace(vals,'Hybrid')
test_df['propulsion_code'] = test_df['propulsion_code'].replace(vals,'Hybrid')


## Outlier Analysis


In [ ]:
#function to plot side by side box plot

def create_box(col):
    
    fig = px.box(train_df, x="accident_severity", y=col,color="sex_of_driver")
    fig.update_traces(quartilemethod="exclusive")
    fig.show()

In [ ]:
# #plotting age of driver against accident severity

# create_box("age_of_driver")

- We can spot outliers in the Slight category for age more than 80. Since there is no upper age limit in the UK to drive a car, we will not remove these but we will transform the variable. We have already seen the distribution is right skewed in the eda stage.

In [ ]:
#log transforming variable 'age_of_driver' on base 10

train_df['age_of_driver'] = np.log10(train_df['age_of_driver'])
test_df['age_of_driver'] = np.log10(test_df['age_of_driver'])

In [ ]:
# #plotting age of vehicle against accident severity
# create_box("age_of_vehicle")

- There are ovious outliers present in the variable age_of_vehicle. As an insurance company we will have to cap these values as too old cars can not be good for the business as it will have more maintenance. So we will cap the values at 30.

In [ ]:
#removing rows  age_of_vehicle > 30

train_df = train_df[train_df['age_of_vehicle'] < 30]
test_df = test_df[test_df['age_of_vehicle'] < 30]



- The variable engine_capacity_cc is also right skewed and we will log transform it to keep the distribution normal.

In [ ]:
#log transforming variable 'engine_capacity_cc' on base 10

train_df['engine_capacity_cc'] = np.log10(train_df['engine_capacity_cc'])
test_df['engine_capacity_cc'] = np.log10(test_df['engine_capacity_cc'])

## Creating Dummy Variables

In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
#putting all categorical columns in one list

cat_cols = [x for x in train_df.select_dtypes("object").columns]
cat_cols

In [ ]:
#Creating dummy variables for all categorical columns
from sklearn.preprocessing import OneHotEncoder
for c in cat_cols:
    
    if c != 'accident_severity': #excluding target variable
        
        #create a nee encoder for each category
        one_hot_encoder = OneHotEncoder(drop='first',handle_unknown='ignore',sparse=False)

        # the input to the encoder must be a 2-d numpy array,
        # so we take the column, extract their values and reshape the array to be 2-d
        cat_vals = train_df[c].values.reshape(-1,1)

        transformed = one_hot_encoder.fit_transform(cat_vals)

        # put the transformed data as columns in the dataframe
        col_names = one_hot_encoder.categories_[0].tolist()[1:]
        for i, col_name in enumerate(col_names):
            train_df[col_name] = transformed[:,i]

        #transforming testset with fitted encoder
        cat_vals = test_df[c].values.reshape(-1,1)
        transformed = one_hot_encoder.transform(cat_vals)

        for i, col_name in enumerate(col_names):
            test_df[col_name] = transformed[:,i]    

        #Delete original categorical columns
        train_df.drop(c,axis=1,inplace=True)
        test_df.drop(c,axis=1,inplace=True)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

## Feature Scaling

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# train_target = train_df["median_house_value"].values
# train_predictors = train_df.drop("median_house_value", axis=1)

# # fit_transform returns a NumPy array, so we need to put it back 
# # into a Pandas dataframe
# scaled_vals = scaler.fit_transform(trainset_predictors)
# trainset = pd.DataFrame(scaled_vals, columns=trainset_predictors.columns)

# # put the non-scaled target back in
# trainset['median_house_value'] = trainset_target

# # inspect the data
# trainset.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#scaling train_df columns

scaler = StandardScaler()

num_cols = [x for x in train_df.select_dtypes("float64").columns ]

train_df[num_cols] = scaler.fit_transform(train_df[num_cols])

test_df[num_cols] = scaler.transform(test_df[num_cols])

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.rename(columns={'Not known': 'Other_gender'}, inplace=True)
test_df.rename(columns={'Not known': 'Other_gender'}, inplace=True)

In [ ]:
#saveing the final data in an excel
#df.to_excel("final_data.xlsx")

In [ ]:
#saveing the final data in an excel
# train_df.to_excel("testset.xlsx")
# test_df.to_excel("trainset.xlsx")

In [ ]:
import io
from nbformat import read, NO_CONVERT

with io.open(NOTEBOOK_FULL_PATH.split("/")[-1], 'r', encoding='utf-8') as f:
    nb = read(f, NO_CONVERT)

word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print(f"Word count: {word_count}")